# Statistik II – Projektarbeit  

### Einordnung und Ziel dieses Notebooks

Dieses Notebook dient der explorativen und modellbasierten Analyse von Protestereignissen anhand der Global Database of Events, Language and Tone (GDELT). Im Fokus steht die Frage, welche Ereignis- und Kontextmerkmale mit der Wahrscheinlichkeit zusammenhängen, dass ein Protestereignis als gewaltsam klassifiziert wird. Die Analyse ist bewusst explorativ und assoziativ angelegt; es werden keine kausalen Effekte identifiziert.

Als Datengrundlage werden GDELT-Ereignisse aus dem Jahr 2019 verwendet, wobei Proteste und Unruhen anhand der CAMEO-Ereignisklassifikation unterschieden werden. Die Gewaltklassifikation basiert somit auf der in GDELT vorgenommenen Einordnung von Protesten (EventRootCode 14) und Unruhen (EventRootCode 18). Ergänzend stehen verschiedene Metadaten zur Verfügung, darunter Angaben zur Konfliktintensität (GoldsteinScale), zur Polizeinennung in den Akteursinformationen sowie zum Land, in dem das Ereignis stattfindet.

GDELT stellt eine Vielzahl weiterer Ereignisattribute zur Verfügung. Variablen wie Teilnehmerzahlen oder konkrete Protestgründe sind jedoch im Event-1.0-Datensatz nicht systematisch und konsistent verfügbar und werden deshalb in dieser Analyse nicht berücksichtigt.

Methodisch kombiniert das Notebook deskriptive Auswertungen mit einer logistischen Regression in Form eines Generalized Linear Models. Aufgrund der binären Zielvariable und der starken Trennung einzelner Prädiktoren wird eine regularisierte Modellvariante eingesetzt, um numerisch stabile Schätzungen zu erhalten. Die Modellresultate werden interpretativ eingeordnet und in Relation zu den zuvor präsentierten deskriptiven Befunden gesetzt.

Das Notebook ist so aufgebaut, dass alle Schritte der Analyse – von der Datenaufbereitung über die explorative Analyse bis zur Modellierung – transparent und reproduzierbar nachvollzogen werden können. Es versteht sich als analytisches Arbeitsdokument und nicht als abschliessende kausale Untersuchung.

Die Analyse stützt sich bewusst auf wenige, zuverlässig verfügbare Metadaten (Land, Konfliktintensität, Polizeinennung, Zeit), da weitergehende Informationen wie Teilnehmerzahlen oder konkrete Protestgründe im GDELT-Event-1.0-Datensatz nicht systematisch und konsistent erfasst sind.


### Verwendete Modellvariablen:

SQLDATE (Zeit)

CountryCode (Land)

GoldsteinScale (Konfliktintensität)

police_mentioned (abgeleitet aus Actor1/Actor2)



### Fragestellung
Welche Ereignis- und Kontextmerkmale stehen in Zusammenhang mit der Wahrscheinlichkeit,
dass ein Protestereignis als gewaltsam klassifiziert wird?



In [1]:
# Grundlegende Datenanalyse
import pandas as pd
import numpy as np

# Visualisierung
import matplotlib.pyplot as plt

# Statistik / Modellierung
import statsmodels.api as sm

# Datenbank für performantes Einlesen
import duckdb
from pathlib import Path


### 2. Pfade definieren

In [2]:
# Basis-Pfad: Projekt-Root
BASE_PATH = Path("..")

# Pfad zu den extrahierten Rohdaten
DATA_DIR = BASE_PATH / "Rohdaten_extrahiert"

print(f"Datenpfad: {DATA_DIR.resolve()}")


Datenpfad: C:\Users\matia\Desktop\statistic_project\Rohdaten_extrahiert


### 3. DuckDB initialisieren

In [3]:
# In-Memory-Datenbank für schnelle Verarbeitung
con = duckdb.connect(database=":memory:")


### 4. Rohdaten einlesen

In [4]:
# Einlesen aller GDELT-CSV-Dateien (ohne Header)
con.execute("""
CREATE OR REPLACE VIEW gdelt_raw AS
SELECT *
FROM read_csv_auto(
    '../Rohdaten_extrahiert/*.CSV',
    delim='\t',
    header=False
)
""")


### 5. Spalten benennen (basierend auf GDELT Field List)

Die folgenden Spalten werden aus dem Rohdatensatz selektiert. Die Zuordnung (z. B. `column01 = SQLDATE`) basiert auf der offiziellen GDELT 1.0 “Event Field List” (Spaltenliste). Es wird bewusst nur ein kleiner, für die Fragestellung relevanter Ausschnitt verwendet.


In [5]:
# Mapping der wichtigsten Spalten (Auszug, bewusst reduziert)
con.execute("""
CREATE OR REPLACE VIEW gdelt_named AS
SELECT
    column01  AS SQLDATE,
    column28  AS EventRootCode,
    column30  AS GoldsteinScale,
    column51  AS CountryCode,
    column05  AS Actor1Code,
    column06  AS Actor1Name,
    column15  AS Actor2Code,
    column16  AS Actor2Name
FROM gdelt_raw
""")

### 6. Zielvariable definieren

In [6]:
# Ableitung der Zielvariable: gewaltsam vs. nicht gewaltsam
# EventRootCode wird robust in Integer umgewandelt
con.execute("""
CREATE OR REPLACE VIEW gdelt_target AS
SELECT
    SQLDATE,
    CountryCode,
    Actor1Code,
    Actor1Name,
    Actor2Code,
    Actor2Name,

    -- Konfliktintensität robust casten
    TRY_CAST(GoldsteinScale AS DOUBLE) AS GoldsteinScale,

    -- Zielvariable explizit als Integer (0/1)
    CAST(
        CASE
            WHEN TRY_CAST(EventRootCode AS INTEGER) = 18 THEN 1
            WHEN TRY_CAST(EventRootCode AS INTEGER) = 14 THEN 0
            ELSE NULL
        END
    AS INTEGER) AS is_violent

FROM gdelt_named
WHERE TRY_CAST(EventRootCode AS INTEGER) IN (14, 18)
""")



### 7. Polizeinennung ableiten (erklärende Variable)

In [7]:
# Polizeinennung über Actor-Codes / Namen
# NULLs werden explizit abgefangen
con.execute("""
CREATE OR REPLACE VIEW gdelt_features AS
SELECT
    SQLDATE,
    CountryCode,
    GoldsteinScale,
    is_violent,

    CAST(
        CASE
            WHEN
                COALESCE(Actor1Name, '') ILIKE '%POLICE%' OR
                COALESCE(Actor2Name, '') ILIKE '%POLICE%' OR
                COALESCE(Actor1Code, '') ILIKE '%POLICE%' OR
                COALESCE(Actor2Code, '') ILIKE '%POLICE%'
            THEN 1
            ELSE 0
        END
    AS INTEGER) AS police_mentioned

FROM gdelt_target
""")



### 8. Validierung der abgeleiteten Polizeinennung

In [8]:
# Verteilung der Polizeinennung
con.execute("""
SELECT
    police_mentioned,
    COUNT(*) AS n_events
FROM gdelt_features
GROUP BY police_mentioned
""").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,police_mentioned,n_events
0,0,1326960
1,1,122146



Die Variable `police_mentioned` bildet ausschliesslich ab, ob Polizeikräfte in den Akteursinformationen eines Ereignisses erwähnt werden. Die zeitliche und kausale Richtung dieser Nennung ist nicht eindeutig bestimmbar: Polizeikräfte können sowohl präventiv präsent sein als auch als Reaktion auf bereits eskalierte Ereignisse auftreten. Entsprechend wird diese Variable im Folgenden ausschliesslich als assoziative Kontextinformation interpretiert.



### 9. Deskriptive Analyse: Polizeinennung × Gewalt

In [9]:
# Kreuztabelle: Polizeinennung und Gewaltklassifikation
# Ziel: Erste deskriptive Einordnung ohne kausale Interpretation
con.execute("""
SELECT
    police_mentioned,
    is_violent,
    COUNT(*) AS n_events
FROM gdelt_features
GROUP BY police_mentioned, is_violent
ORDER BY police_mentioned, is_violent
""").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,police_mentioned,is_violent,n_events
0,0,0,538884
1,0,1,788076
2,1,0,33810
3,1,1,88336


Die dargestellten Zusammenhänge sind rein deskriptiv zu interpretieren und erlauben keine Rückschlüsse auf kausale Wirkungen der Polizeipräsenz.


### 10. Deskriptive Analyse: Gewaltanteil nach Polizeinennung

In [10]:
# Anteil gewaltsamer Ereignisse nach Polizeinennung
# Ziel: Prüfung einer deskriptiven Assoziation
con.execute("""
SELECT
    police_mentioned,
    AVG(is_violent) AS violence_share,
    COUNT(*) AS n_events
FROM gdelt_features
GROUP BY police_mentioned
""").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,police_mentioned,violence_share,n_events
0,0,0.593896,1326960
1,1,0.723200,122146


### 11. Deskriptive Analyse: Gewaltanteil nach Polizeinennung

In [11]:
# Gewaltanteil nach Ländern (nur Länder mit ausreichender Fallzahl)
# Ziel: Aufzeigen von Kontextunterschieden jenseits der Polizeinennung
con.execute("""
SELECT
    CountryCode,
    COUNT(*) AS n_events,
    AVG(is_violent) AS violence_share
FROM gdelt_features
GROUP BY CountryCode
HAVING COUNT(*) > 500
ORDER BY violence_share DESC
""").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,CountryCode,n_events,violence_share
0,DR,846,0.905437
1,UV,1339,0.887229
2,CD,732,0.848361
3,SO,3772,0.839343
4,RW,1401,0.834404
...,...,...,...
131,KZ,1113,0.203953
132,SU,14878,0.189743
133,GG,1558,0.173941
134,HK,17757,0.163597


### 12. Deskriptive Analyse: Konfliktintensität (GoldsteinScale)

In [12]:
# Vergleich der durchschnittlichen Konfliktintensität nach Gewaltstatus
# Ziel: Abgrenzung zwischen Klassifikation (Gewalt) und Intensität
con.execute("""
SELECT
    is_violent,
    AVG(GoldsteinScale) AS avg_goldstein,
    COUNT(*) AS n_events
FROM gdelt_features
WHERE GoldsteinScale IS NOT NULL
GROUP BY is_violent
""").fetchdf()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,is_violent,avg_goldstein,n_events
0,0,-6.589192,572694
1,1,-9.194742,876412


Die GoldsteinScale misst die Intensität politischer Ereignisse und ist konzeptionell eng mit Eskalationsprozessen verbunden. Da die Zielvariable auf der Unterscheidung zwischen Protesten und Unruhen basiert, kann diese Variable inhaltlich nahe an der Gewaltklassifikation liegen. Im Modellteil wird dieser Aspekt explizit reflektiert, unter anderem durch eine ergänzende Robustheitsanalyse ohne GoldsteinScale.


# 13. Definition der finalen Modellbasis

In [13]:
# Erstellung der finalen Modellbasis
# Alle Variablen sind jetzt numerisch und modellbereit
con.execute("""
CREATE OR REPLACE VIEW gdelt_model_base AS
SELECT
    is_violent,           -- Zielvariable (0/1)
    police_mentioned,     -- Akteurskonstellation
    GoldsteinScale,       -- Konfliktintensität
    CountryCode           -- Kontextvariable
FROM gdelt_features
WHERE
    SQLDATE BETWEEN 20190101 AND 20191231
    AND GoldsteinScale IS NOT NULL
    AND is_violent IS NOT NULL
""")


### 14. Stichprobenreduktion für das Regressionsmodell

Aufgrund der sehr grossen Stichprobe und der hohen Dimensionalität durch Länderdummies wurde für die Regressionsschätzung eine stratifizierte Zufallsstichprobe gezogen, um eine numerisch stabile Schätzung zu ermöglichen.

In [14]:
# Stichprobenreduktion zur numerisch stabilen Schätzung des GLM
# Ziel: Reduktion der Beobachtungen bei Erhalt der Klassenstruktur

con.execute("""
CREATE OR REPLACE VIEW gdelt_model_sample AS
SELECT *
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY is_violent ORDER BY random()) AS rn
    FROM gdelt_model_base
)
WHERE
    (is_violent = 0 AND rn <= 50000)
 OR (is_violent = 1 AND rn <= 50000)
""")


### 15. Übergang zu pandas

In [15]:
# Übergang zu pandas für das statistische Modell
# Die Daten sind nun vollständig modellbereit
df = con.execute("""
SELECT *
FROM gdelt_model_sample
""").fetchdf()

df.info()



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   is_violent        100000 non-null  int32  
 1   police_mentioned  100000 non-null  int32  
 2   GoldsteinScale    100000 non-null  float64
 3   CountryCode       95741 non-null   object 
 4   rn                100000 non-null  int64  
dtypes: float64(1), int32(2), int64(1), object(1)
memory usage: 3.1+ MB


# Generalized Linear Model (GLM) mit binomialer Verteilung und Logit-Linkfunktion

### Designmatrix bauen

In [16]:
# -----------------------------------------
# 16. Designmatrix für das GLM
# -----------------------------------------

# Zielvariable
y = df["is_violent"].astype(int)

# Numerische Prädiktoren
X = df[["police_mentioned", "GoldsteinScale"]].copy()

# -----------------------------------------
# Länder als Dummy-Variablen (vollständig)
# -----------------------------------------

country_dummies = pd.get_dummies(
    df["CountryCode"],
    drop_first=True
).astype(int)

# Designmatrix zusammenbauen
X = pd.concat([X, country_dummies], axis=1)

# Intercept hinzufügen
X = sm.add_constant(X)


Die Designmatrix umfasst ausgewählte Ereignis- und Kontextmerkmale, welche zuverlässig aus den GDELT-Daten ableitbar sind. Länder werden als Kontrollvariablen berücksichtigt. Die Zielvariable basiert ausschliesslich auf der GDELT-Ereignisklassifikation und wird nicht als erklärende Variable in das Modell aufgenommen.


### Gewichtung

Da für die Modellschätzung eine stratifizierte Stichprobe mit gleicher Anzahl gewaltsamer und nicht-gewaltsamer Ereignisse gezogen wurde (50’000 pro Klasse), ist das Klassenungleichgewicht in der Modellstichprobe bereits ausgeglichen. Eine zusätzliche Gewichtung ist daher nicht notwendig.


In [17]:
# Keine Gewichtung notwendig, da balancierte Modellstichprobe (50k/50k)
w = np.ones(len(y))


### GLM schätzen

In [18]:
# -----------------------------
# 18. GLM schätzen (regularisiert)
# -----------------------------

model = sm.GLM(
    y,
    X,
    family=sm.families.Binomial(),
    freq_weights=w
)

# Regularisierte logistische Regression (Elastic Net)
# L1_wt = 1.0 entspricht reiner Lasso-Regularisierung
result = model.fit_regularized(
    method="elastic_net",
    alpha=0.1,
    L1_wt=1.0,
    maxiter=100
)


Aufgrund starker Trennung zwischen gewaltsamen und nicht-gewaltsamen Ereignissen in einzelnen Prädiktoren (Separationsproblem) wird eine regularisierte logistische Regression geschätzt. Diese liefert stabile Koeffizienten und führt gleichzeitig eine implizite Variablenselektion durch.


### Modell-Output anzeigen

In [19]:
result.params



const               0.000000
police_mentioned    0.000000
GoldsteinScale     -0.034691
AC                  0.000000
AE                  0.000000
                      ...   
WZ                  0.000000
YI                  0.000000
YM                  0.000000
ZA                  0.000000
ZI                  0.000000
Length: 234, dtype: float64

In [20]:
result.params[result.params != 0]

GoldsteinScale   -0.034691
dtype: float64

# Interpretation des regularisierten logistischen Modells mit GoldsteinScale

Zur Analyse der Determinanten gewaltsamer Protestereignisse wurde ein regularisiertes Generalized Linear Model mit binärer Zielvariable geschätzt. Die Zielvariable klassifiziert Ereignisse als gewaltsam oder nicht gewaltsam auf Basis der GDELT-Ereignisklassifikation. Als erklärende Variablen wurden die Konfliktintensität (GoldsteinScale), eine binäre Variable zur Polizeinennung sowie Länder-Dummyvariablen berücksichtigt. Aufgrund starker Separationsprobleme in den Daten wurde eine L1-regularisierte logistische Regression (Elastic Net) verwendet, um stabile und wohldefinierte Koeffizienten zu erhalten.

Die Modellergebnisse zeigen, dass einzig die GoldsteinScale einen eigenständigen Erklärungsbeitrag zur Gewaltklassifikation liefert. Der negative Koeffizient der GoldsteinScale weist darauf hin, dass Ereignisse mit stärker negativer Konfliktintensität eine höhere Wahrscheinlichkeit aufweisen, als gewaltsam klassifiziert zu werden. Dieses Ergebnis ist konsistent mit der inhaltlichen Bedeutung der GoldsteinScale als aggregierte Intensitätsmetrik politischer Ereignisse.

Die Variable zur Polizeinennung sowie die Länder-Dummyvariablen werden durch die Regularisierung auf null gesetzt. Dies deutet darauf hin, dass diese Variablen keinen zusätzlichen Erklärungsgehalt liefern, sobald die Konfliktintensität berücksichtigt wird. Polizeipräsenz und Länderunterschiede wirken demnach primär indirekt über das allgemeine Konfliktniveau eines Ereignisses und nicht als eigenständige Faktoren der Gewaltklassifikation.

### Robustheitsanalyse: Modell ohne GoldsteinScale

In [ ]:
# -----------------------------------------
# Robustheitsanalyse: Modell ohne GoldsteinScale
#Dieses Modell dient ausschliesslich der Robustheitsprüfung und nicht der inhaltlichen Hauptinterpretation.

# -----------------------------------------

y2 = df["is_violent"].astype(int)

X2 = df[["police_mentioned"]].copy()

# Länder wieder als Kontrolle
country_dummies_2 = pd.get_dummies(
    df["CountryCode"],
    drop_first=True
).astype(int)

X2 = pd.concat([X2, country_dummies_2], axis=1)
X2 = sm.add_constant(X2)

model_wo_goldstein = sm.GLM(
    y2,
    X2,
    family=sm.families.Binomial()
)

result_wo_goldstein = model_wo_goldstein.fit_regularized(
    method="elastic_net",
    alpha=0.1,
    L1_wt=1.0,
    maxiter=100
)

result_wo_goldstein.params[result_wo_goldstein.params != 0]


Series([], dtype: float64)

# Interpretation des ergänzenden Modells ohne GoldsteinScale

In einem zweiten Schritt wurde ein ergänzendes Modell ohne GoldsteinScale geschätzt, um zu untersuchen, inwiefern Polizeinennung und Länderunterschiede auch ohne direkte Kontrolle der Konfliktintensität mit der Gewaltklassifikation in Zusammenhang stehen. Das Modell wurde analog als regularisierte logistische Regression geschätzt, um Separationsprobleme und instabile Koeffizienten zu vermeiden.

In diesem Modell werden sämtliche erklärenden Variablen durch die Regularisierung auf null gesetzt. Dies zeigt, dass weder die Polizeinennung noch Länderunterschiede einen stabilen eigenständigen Zusammenhang mit der Gewaltklassifikation aufweisen, wenn die konfliktspezifische Intensitätsmetrik ausgeblendet wird. Die Ergebnisse legen nahe, dass beobachtete Zusammenhänge zwischen Polizeipräsenz, Ländern und Gewalt primär über die Konfliktintensität vermittelt werden und nicht als robuste direkte Effekte interpretierbar sind.